In [1]:
import pandas as pd

In [6]:
path = '/relation/'
dis_dis = pd.read_csv(path+'Di_Di_res.csv')
dis_gene = pd.read_csv(path+'Di_G_res.csv')
dis_sys = pd.read_csv(path+'Di_Sy_res.csv')
gene_gene = pd.read_csv(path+'Gene_Gene.csv')

In [1]:
# Load all relationship data
dis_dis = pd.read_csv("/Users/weitingwang/Desktop/@Research/Network Analysis/relation/Di_Di_res.csv")
dis_gene = pd.read_csv("/Users/weitingwang/Desktop/@Research/Network Analysis/relation/Di_G_res.csv")
dis_sys = pd.read_csv("/Users/weitingwang/Desktop/@Research/Network Analysis/relation/Di_Sy_res.csv")
gene_gene = pd.read_csv("/Users/weitingwang/Desktop/@Research/Network Analysis/relation/Gene_Gene.csv")

print("Original shapes:")
print(f"dis_dis: {dis_dis.shape}")
print(f"dis_gene: {dis_gene.shape}")
print(f"dis_sys: {dis_sys.shape}")
print(f"gene_gene: {gene_gene.shape}")


# --- Remove Inferred relationship ---

# Remove Inferred_Relation from gene_gene
if 'Inferred_Relation' in gene_gene.columns:
    gene_gene = gene_gene.drop(columns=['Inferred_Relation'])
    print("Removed Inferred_Relation from gene-gene")

# Remove Inferred_Relation from dis_gene
if 'Inferred_Relation' in dis_gene.columns:
    dis_gene = dis_gene.drop(columns=['Inferred_Relation'])
    print("Removed Inferred_Relation from disease-gene")

print(f"Final shapes - Gene-gene: {gene_gene.shape}, Disease-gene: {dis_gene.shape}")

In [ ]:
# Convert to uppercase
dis_dis['Disease_1'] = dis_dis['Disease_1'].str.upper()
dis_dis['Disease_2'] = dis_dis['Disease_2'].str.upper()
dis_gene['Disease'] = dis_gene['Disease'].str.upper()
dis_gene['Gene'] = dis_gene['Gene'].str.upper()
dis_sys['Disease'] = dis_sys['Disease'].str.upper()
dis_sys['Symptom'] = dis_sys['Symptom'].str.upper()
gene_gene['Gene_1'] = gene_gene['Gene_1'].str.upper()
gene_gene['Gene_2'] = gene_gene['Gene_2'].str.upper()

# Standardize column names
dis_dis_1 = dis_dis[['Disease_1','Disease_2']].rename(columns={'Disease_1':'source','Disease_2':'edge'})
dis_sys_1 = dis_sys[['Disease','Symptom']].rename(columns={'Disease':'source','Symptom':'edge'})
dis_gene_1 = dis_gene[['Disease','Gene']].rename(columns={'Disease':'source','Gene':'edge'})
gene_gene_1 = gene_gene[['Gene_1','Gene_2']].rename(columns={'Gene_1':'source','Gene_2':'edge'})

# Combine all relationships
relation_df = pd.concat([dis_dis_1, dis_gene_1, dis_sys_1, gene_gene_1])

print(f"\nCombined relationships: {relation_df.shape}")
print(relation_df.head())

# Save standardized data
relation_df.to_csv("processed_relationships.csv", index=False)
print("\nSaved as: processed_relationships.csv")

In [ ]:
# Load entity vocabulary data
path = '/entity/'
dis_df = pd.read_csv(path+'disease_vocab.csv')
gene_df = pd.read_csv(path+'gene_vocab.csv')
sym_df = pd.read_csv(path+'symptom_vocab.csv')

print("Original shapes:")
print(f"disease: {dis_df.shape}")
print(f"gene: {gene_df.shape}")
print(f"symptom: {sym_df.shape}")

In [ ]:
# Convert to uppercase
dis_df['primary'] = dis_df['primary'].str.upper()
gene_df['primary'] = gene_df['primary'].str.upper()
sym_df['primary'] = sym_df['primary'].str.upper()

# Check entity types
print('\nEntity types:')
print('disease:', dis_df['primary'].str.split(':').str[0].value_counts())
print('gene:', gene_df['primary'].str.split(':').str[0].value_counts())
print('symptom:', sym_df['primary'].str.split(':').str[0].value_counts())

In [ ]:
# Combine disease and symptom for duplicate checking
dis_all = pd.concat([dis_df, sym_df])
duplicates = dis_all[dis_all['primary'].duplicated()]

print(f"\nDuplicates found: {len(duplicates)}")

# Remove duplicates and process each entity type
dis_df_remove_dup = dis_df[~dis_df['primary'].isin(duplicates['primary'])]
sym_df_remove_umls = sym_df[~sym_df['primary'].str.contains('UMLS')]  # no umls code in dis-sys
gene_df_drop_dup = gene_df.drop_duplicates(subset=['primary']).rename(columns={'symbol': 'name'})

# Create standardized entity datasets
dis_df_1 = dis_df_remove_dup.assign(category='disease')[['primary','name','category']]
gene_df_1 = gene_df_drop_dup.assign(category='gene')[['primary','name','category']]
sym_df_1 = sym_df_remove_umls.assign(category='symptom')[['primary','name','category']]

# Combine all entities
entity_df = pd.concat([dis_df_1, gene_df_1, sym_df_1]).rename(columns={'primary':'node_id'})

print(f"\nFinal entity dataset: {entity_df.shape}")
print(f"Categories: {entity_df['category'].value_counts().to_dict()}")

# Save processed entity data
entity_df.to_csv("processed_entities.csv", index=False)
print("\nSaved as: processed_entities.csv")